In [1]:
# Setup the Dash version
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import urllib.parse

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = urllib.parse.quote_plus("aacuser")
password = urllib.parse.quote_plus("SNHU1234")  # Replace with your password
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Module 5 Assignment - Amaro Terrazas'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        page_size=10,
        sort_action="native",
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    html.H3("Dashboard by Amaro Terrazas")
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:  # Handle empty data
        return [html.P("No data available for mapping.")]
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
    # Default coordinates for Austin, TX if indices are invalid
    try:
        lat = float(dff.iloc[row, 13]) if len(dff.columns) > 13 else 30.75
        lon = float(dff.iloc[row, 14]) if len(dff.columns) > 14 else -97.48
        breed = dff.iloc[row, 4] if len(dff.columns) > 4 else "Unknown"
        name = dff.iloc[row, 9] if len(dff.columns) > 9 else "Unknown"
    except (ValueError, IndexError):
        lat, lon = 30.75, -97.48
        breed = "Unknown"
        name = "Unknown"
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),  # Breed
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(name)  # Name
                        ])
                    ]
                )
            ]
        )
    ]

app.run_server(port=8050, debug=False)

Connection to MongoDB successful
